## 작업형 2 예시
- 데이터 가져오기
    + [고객지원] - [공지사항] - 제2회 빅데이터분석기사 실기 안내(수정: 체험링크 추가)

## 문제
- 아래는 백화점 고객의 1년 간 구매 데이터이다.
- 제공 데이터 목록
  + y_train.csv : 고객의 성별 데이터(학습용, CSV 형식의 파일)
  + X_train.csv, X_test.csv : 고객의 상품구매 속성(학습용 및 평가용), CSV 형식의 파일
- 데이터 형식 및 내용
  + y_train.csv (3,500명 데이터)
    * custid : 고객 ID
    * gender: 고객의 성별(0: 여자, 1:남자)
- 고객 3,5000명에 대한 학습용 데이터를 이용하여 성별예측 모형을 만드 후, 이를 평가용 데이터에 적용하여 얻은 고객의 성별 예측값(남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성
- 제출한 모델의 성능은 ROC-AUC 평가지표에 따라 채점

In [1]:
import pandas as pd 
pd.DataFrame({
    'custid' : [3500, 3501, 3502], 
    'gender' : [0.267, 0.578, 0.885]
})

,custid,gender
0,3500,0.267
1,3501,0.578
2,3502,0.885


In [2]:
import pandas as pd

DATA_PATH = 'data/department_store/'

X_test = pd.read_csv(DATA_PATH + "X_test.csv", encoding="euc-kr")   # 작업장 환경에서는 불필요
X_train = pd.read_csv(DATA_PATH + "X_train.csv", encoding="euc-kr")
y_train = pd.read_csv(DATA_PATH + "y_train.csv", encoding="euc-kr")

## 데이터확인

In [3]:
print(X_train.head())
print(y_train.head())

   cust_id      총구매액     최대구매액       환불금액   주구매상품 주구매지점  내점일수   내점당구매건수  \
0        0  68282840  11264000  6860000.0      기타   강남점    19  3.894737   
1        1   2136000   2136000   300000.0     스포츠   잠실점     2  1.500000   
2        2   3197000   1639000        NaN  남성 캐주얼   관악점     2  2.000000   
3        3  16077620   4935000        NaN      기타   광주점    18  2.444444   
4        4  29050000  24000000        NaN      보석  본  점     2  1.500000   

     주말방문비율  구매주기  
0  0.527027    17  
1  0.000000     1  
2  0.000000     1  
3  0.318182    16  
4  0.000000    85  
   cust_id  gender
0        0       0
1        1       0
2        2       1
3        3       1
4        4       0


In [4]:
print(X_train.info())
print()
print(y_train.info())
print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   cust_id  3500 non-null   int64  
 1   총구매액     3500 non-null   int64  
 2   최대구매액    3500 non-null   int64  
 3   환불금액     1205 non-null   float64
 4   주구매상품    3500 non-null   object 
 5   주구매지점    3500 non-null   object 
 6   내점일수     3500 non-null   int64  
 7   내점당구매건수  3500 non-null   float64
 8   주말방문비율   3500 non-null   float64
 9   구매주기     3500 non-null   int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 273.6+ KB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3500 entries, 0 to 3499
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   cust_id  3500 non-null   int64
 1   gender   3500 non-null   int64
dtypes: int64(2)
memory usage: 54.8 KB
None



In [5]:
print(X_train.isnull().sum())
print()
print(y_train.isnull().sum())
print()
print(X_test.isnull().sum())
print()

cust_id       0
총구매액          0
최대구매액         0
환불금액       2295
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64

cust_id    0
gender     0
dtype: int64

cust_id       0
총구매액          0
최대구매액         0
환불금액       1611
주구매상품         0
주구매지점         0
내점일수          0
내점당구매건수       0
주말방문비율        0
구매주기          0
dtype: int64



In [6]:
X_train['환불금액'] = X_train['환불금액'].fillna(X_train['환불금액'].median())
print(X_train.isnull().sum())

X_test['환불금액'] = X_test['환불금액'].fillna(X_test['환불금액'].median())
print(X_test.isnull().sum())

cust_id    0
총구매액       0
최대구매액      0
환불금액       0
주구매상품      0
주구매지점      0
내점일수       0
내점당구매건수    0
주말방문비율     0
구매주기       0
dtype: int64
cust_id    0
총구매액       0
최대구매액      0
환불금액       0
주구매상품      0
주구매지점      0
내점일수       0
내점당구매건수    0
주말방문비율     0
구매주기       0
dtype: int64


## 데이터셋 분리

In [7]:
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
print(X_tr.shape, X_te.shape, y_tr.shape, y_te.shape)

(2800, 10) (700, 10) (2800, 2) (700, 2)


## 컬럼 삭제 및 분리

In [8]:
X_tr.pop('cust_id')
X_te.pop('cust_id')
y_tr.pop('cust_id')
y_te.pop('cust_id')

print(X_tr.shape, X_te.shape, y_tr.shape, y_te.shape)

X_test_id = X_test.pop('cust_id')

(2800, 9) (700, 9) (2800, 1) (700, 1)


## 데이터 타입별로 분리

In [9]:
import numpy as np

object_df = X_tr.select_dtypes(include=object)
number_df = X_tr.select_dtypes(include=np.number)

In [10]:
for column in object_df.columns:
    print(object_df[column].value_counts())

기타        475
가공식품      433
농산물       277
화장품       204
시티웨어      169
디자이너      151
수산품       123
명품         85
섬유잡화       84
캐주얼        79
골프         58
스포츠        53
일용잡화       51
모피/피혁      51
육류         48
남성 캐주얼     44
구두         43
차/커피       36
피혁잡화       35
건강식품       34
축산가공       31
아동         29
주방용품       27
셔츠         25
젓갈/반찬      25
트래디셔널      21
주방가전       19
남성정장       17
생활잡화       13
주류         10
가구          9
커리어         6
란제리/내의      6
식기          5
대형가전        5
액세서리        5
침구/수예       3
보석          3
통신/컴퓨터      3
악기          2
소형가전        2
남성 트랜디      1
Name: 주구매상품, dtype: int64
본  점     851
잠실점      385
분당점      359
부산본점     196
영등포점     194
일산점      156
강남점      114
광주점       96
노원점       74
청량리점      70
대전점       55
부평점       48
미아점       48
동래점       39
관악점       33
안양점       25
인천점       25
포항점        8
울산점        6
대구점        6
센텀시티점      5
전주점        3
창원점        3
상인점        1
Name: 주구매지점, dtype: int64


In [11]:
print(number_df.describe())

               총구매액         최대구매액          환불금액         내점일수      내점당구매건수  \
count  2.800000e+03  2.800000e+03  2.800000e+03  2800.000000  2800.000000   
mean   9.252644e+07  1.980003e+07  1.365154e+07    19.462500     2.847587   
std    1.638971e+08  3.037693e+07  3.054602e+07    27.751436     1.932191   
min   -5.242152e+07 -2.992000e+06  1.800000e+04     1.000000     1.000000   
25%    4.827700e+06  2.898000e+06  7.392000e+06     2.000000     1.700000   
50%    2.837486e+07  9.875000e+06  7.392000e+06     8.000000     2.333333   
75%    1.068782e+08  2.316600e+07  7.392000e+06    26.000000     3.400000   
max    2.323180e+09  3.910000e+08  5.637530e+08   285.000000    22.083333   

            주말방문비율         구매주기  
count  2800.000000  2800.000000  
mean      0.304402    20.746071  
std       0.288487    24.460119  
min       0.000000     0.000000  
25%       0.022480     4.000000  
50%       0.252339    13.000000  
75%       0.444444    28.000000  
max       1.000000   162.000000  


## 모델 생성

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from lightgbm import LGBMClassifier

transformer = ColumnTransformer([
    ('scaler', RobustScaler(), number_df.columns),
    ('encoder', OneHotEncoder(), object_df.columns)
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', transformer),
    ('model', LGBMClassifier(random_state=42, max_depth=3, learning_rate=0.025))
])

pipeline.fit(X_tr, y_tr)

D:\Anaconda\Lib\site-packages\sklearn\preprocessing\_label.py:97: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\Lib\site-packages\sklearn\preprocessing\_label.py:132: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', RobustScaler(),
                                                  Index(['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기'], dtype='object')),
                                                 ('encoder', OneHotEncoder(),
                                                  Index(['주구매상품', '주구매지점'], dtype='object'))])),
                ('model',
                 LGBMClassifier(learning_rate=0.025, max_depth=3,
                                random_state=42))])

In [16]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier

transformer = ColumnTransformer([
    ('scaler', RobustScaler(), number_df.columns),
    ('encoder', OneHotEncoder(), object_df.columns)
], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', transformer),
    ('model', RandomForestClassifier(random_state=42, n_estimators=100, max_depth=3))
])

pipeline.fit(X_tr, y_tr)

D:\Anaconda\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scaler', RobustScaler(),
                                                  Index(['총구매액', '최대구매액', '환불금액', '내점일수', '내점당구매건수', '주말방문비율', '구매주기'], dtype='object')),
                                                 ('encoder', OneHotEncoder(),
                                                  Index(['주구매상품', '주구매지점'], dtype='object'))])),
                ('model',
                 RandomForestClassifier(max_depth=3, random_state=42))])

## 모델 평가

In [17]:
from sklearn.metrics import roc_auc_score

def get_scores(model, X_tr, X_te, y_tr, y_te):
    train_predict = model.predict_proba(X_tr)
    test_predict = model.predict_proba(X_te)
    train_score = roc_auc_score(y_tr, train_predict[:, 1])
    test_score = roc_auc_score(y_te, test_predict[:, 1])    
    return f"train : {train_score}, test : {test_score}"

print(get_scores(pipeline, X_tr, X_te, y_tr, y_te))

train : 0.690090753272351, test : 0.6809583858764185


## 결과제출

In [18]:
final_predict = pipeline.predict_proba(X_test)
result = pd.DataFrame({
    'cust_id' : X_test_id,
    'gender' : final_predict[:, 1]
})
print(result)

      cust_id    gender
0        3500  0.401842
1        3501  0.247428
2        3502  0.249309
3        3503  0.384044
4        3504  0.403472
...       ...       ...
2477     5977  0.370295
2478     5978  0.451998
2479     5979  0.370941
2480     5980  0.348606
2481     5981  0.424711

[2482 rows x 2 columns]
